<br>

<img src="https://www.fedlex.admin.ch/assets/pictos/logo-ch.png" style="width:15%; float:right">

# Fedlex Linked Data Tutorial
**Version vom 12.05.2023 - Work in Progress**

## Einführung

Dieses Tutorial ist dazu gedacht, eine Einführung in Fedlex Linked Data zu geben. Dabei soll einerseits das verwendete Datenmodell vorgestellt werden und anderseits aufgezeigt werden, wie mit Hilfe von SPARQL Abfragen die Daten genutzt werden können. Fedlex ist die Publikationsplattform des schweizerischen Bundesrechts und baut auf Linked Data auf.

## Zielpublikum

Das Zielpublikum dieses Tutorials sind Personen, die beruflich mit Fedlex Daten zu tun haben und über gute Grundkenntnisse in der Informatik verfügen, die aber noch wenig Knowhow zum Thema Linked Data und SPARQL besitzen.

## Interaktives Notebook

Dieses Tutorial ist ein sogenanntes **interaktives JupyterLite Notebook**. In diesem Notebook können Sie den Inhalt der einzelnen Zellen interaktiv ändern und diese Zellen direkt ausführen, um das Ergebnis Ihrer Änderungen sofort zu sehen. Die Zellen enthalten entweder [Markdown](https://en.wikipedia.org/wiki/Markdown)-Inhalt (wie diese Zelle) oder ausführbaren Python-Quellcode. Dies ist für ein Tutorial sehr hilfreich, weil Inhalte beliebig mit ausführbarem Quellcode kombiniert werden können. Es können also Abfragen gezeigt werden, diese erklärt werden und darauf weiter aufgebaut werden.

**Um direkt loslegen zu können klicken Sie oben im Menu auf Run -> Run All Cells.**  
**Einzelne ausgewählte Zellen können sie danach abändern und mit dem "Play-Button" erneut ausführen und so Abfragen individuell anpassen.**

Das Notebook startet mit einem [Setup](#Setup) der Programierumgebung. Das eigentliche Tutorial startet [hier](#Tutorial).

*Zusätzliche Informationen zu JupyterLite:*  
JupyterLite is ein spezielles Jupyter Notebook mit dem Vorteil, vollständig browserbasiert zu sein, ohne eine aufwändige Backend-Infrastruktur zu benötigen. Der Nachteil ist, dass die erstmalige Ausführung der Zellen einige Zeit in Anspruch nehmen kann, weil eine erhebliche Menge von Daten geladen werden muss. Dass eine Zelle noch in Ausführung ist, ist am `[*]` links neben der Zelle erkennbar. Nach Abschluss der Ausführung erscheint statt eines `*` eine Zahl. Vor der ersten Ausführung ist eine leere Klammer `[ ]` zu sehen. Nachfolgende wiederholte Ausführungen von Zellen werden aufgrund der gespeicherten Daten in Ihrem Browser-Cache viel schneller sein. Wenn Sie Änderungen am Tutorial vornehmen, werden diese Änderungen automatisch lokal im Browser-Cache gespeichert. Bei einem darauffolgenden Öffnen werden wiederum die Änderungen aus dem Browser-Cache geladen. Um wieder zur Ursprüngsversion des Tutorials zurückzukehren, müssen entweder die Browser-Daten gelöscht werden, oder Sie öffnen das Tutorial in einem Chrome-Inkognito Fenster, wo die Änderungen nach Schliessen des Fensters aus dem Cache gelöscht werden.

Wenn Sie mehr über die Handhabung von Jupyter Notebooks wissen wollen, finden Sie hier zwei nützliche Ressourcen:

- [Die JupyterLab Interface](https://jupyterlab.readthedocs.io/en/stable/user/interface.html)
- [Das Jupyter Notebook](https://jupyterlab.readthedocs.io/en/stable/user/notebook.html)

## Setup

Eine SPARQL Abfrage ist nichts anderes als ein POST-Request an den entsprechenden Triple Store Datenbank Server. Um diese Requests und die erhaltenen Antworten einfacher handhaben zu können, enthält dieses Notebook vorbereiteten Python Code, der nachfolgend importiert wird. Zusätzlich wird das `pandas` Modul importiert, welches die Möglichkeit bietet, die tabellarischen Daten der SPARQL Abfragen als [Pandas Dataframes](https://pandas.pydata.org/docs/index.html) weiter zu verarbeiten. 

In [1]:
import pandas as pd
from ext.sparql import query, display_result

# Linked Data Einführung

Linked Data beschreibt ein **Framework für den Umgang mit Daten**, die sowohl für Menschen nützlich sein sollen, als auch maschinenlesbar sind inkl. einer von Computern verarbeitbaren Semantik. Also sowohl Menschen als auch Computer sollen die Daten "verstehen" und interpretieren können. 

## RDF

Das für Linked Data verwendete Datenformat ist RDF (Resource Description Framework). Das bedeutet, dass die Daten nicht als Tabellen (wie beispielsweise in relationalen Datenbanken) sondern als **Triples** abgespeichert werden. Triples folgen der grammatikalischen Struktur **Subjekt -> Prädikat -> Objekt** und können auch als grammatikalischer Satz verstanden werden. 

Die Information "**Der Apfel ist grün**" wird also mit dem Tripel **Apfel -> ist -> grün** ausgedrückt. Alle Teile eines Triples sind dabei durch weitere Eigenschaften definiert und beschrieben die wiederum in Form von Triples beschrieben sind. Diese vielseitigen Verknüpfungen führen zu einer Netzwerkstruktur, zu einem sogenannten Graphen.

Nachfolgendes Bild aus dem [W3C Primer für RDF](https://www.w3.org/TR/rdf11-primer/) veranschaulicht diese Zusammenhänge:

![](https://www.w3.org/TR/rdf11-primer/example-graph.jpg)

## URI

Eine weitere wichtige Eigenschaft von Linked Data ist, dass die Teile eines Triples, also Subjekt, Prädikat und Objekt weltweit eindeutig identifizierbar sind. Dafür werden **URI** (Universal Resource Identifier) eingesetzt. Die URI https://fedlex.data.admin.ch/eli/cc/1999/404 beispielsweise ist der weltweit eindeutige Identifier für die Bundesverfassung in der systematischen Rechtssammlung. Typischerweise lassen sich URI **dereferenzieren**, das heisst, ein Request auf die entsprechende URI (bspw. in dem man sie in die Adresszeile eines Browsers eintippt) führt zu einer Website, die Infos zur entsprechenden URI enthält. Im Falle der URI der Bundesverfassung wird man auf eine Webpage weitergeleitet, die diverse Informationen zur Bundesverfassung (Chronologie, Änderungen, etc.) und den eigentlichen Text der Bundesverfassung enthält. Während Prädikate immer URIs sind, können die Objekte auch sogeannte **Literals** sein, also reine Stringwerte (Zahlen und Buchstaben), die eine Information transportieren, aber die nicht dereferenzierbar sind. Solche Literals können beispielsweise Kalenderdaten sein oder Identifier, Namen, etc.

## SPARQL

SPARQL ist eine Query-Sprache für Linked Data Triple Stores ähnlich wie es SQL für relationale Datenbanken ist. Mit Hilfe von SPARQL können Daten nicht nur abgefragt werden, sondern diese können auch gleich weiter aufbereitet werden: Resultate können bspw. gruppiert, summiert oder gemittelt werden. SPARQL Abfragen (engl. Queries) können entweder direkt über ein eigenes Web-Interface eingegeben werden oder als HTTP-POST Request an einen sogenannten SPARQL-Endpoint geschickt werden.

Für die Fedlex Daten gelten folgende URLs:

- [SPARQL Web-Interface von Fedlex](https://fedlex.data.admin.ch/de-CH/sparql)
- SPARQL-Endpoint: https://fedlex.data.admin.ch/sparqlendpoint

Die HTTP-POST Requests Methode erlaubt es, eigene Anwendungen zu bauen, die automatisch aktuelle Daten von Fedlex abfragen können. Für dieses Tutorial verwenden wir diese Methode. Die eigentlichen Abfragen sind jedoch in beiden Fällen identisch.

### Pattern Matching

SPARQL Queries sind Aufträge an den Computer, bestimme Muster (Pattern) in den Daten zu finden (matching). Es können also mit Hilfe von SPARQL Muster vorgegeben werden, und die Datenbank gibt alle Triples zurück, die dieses Muster erfüllen. Einzelne Positionen der Triples können dabei bei einer Abfrage bewusst undefiniert gelassen und mit einer Variable bezeichnet werden. Variablen beginnen mit `?` und werden bei der Abfrage durch alle möglichen Elemente gefüllt, die dieses Pattern erfüllen. 

Eine ausführlicher Anleitung zum Pattern Matching ist [hier](https://programminghistorian.org/en/lessons/retired/graph-databases-and-SPARQL#rdf-in-brief) zu finden. 

### Aufbau von SPARQL Queries

SPARQL Queries folgen einem grundlegenden Aufbau, der immer gleich lautet:

```
SELECT * WHERE 
{
    ?subject ?predicate ?object.   
}
```

Nach dem Schlüsselwort `SELECT` folgen die Variablen, die als Resultat angezeigt werden sollen, ein `*` bedeutet, dass alle in der Query vorkommenden Variablen angezeigt werden sollen. Nach dem Schlüsselwort `WHERE` und innerhalb von  geschweiften Klammern `{}` folgen nun die einzelnen Pattern, die das zurückgegebene Resultat erfüllen soll. Jedes Pattern wird dabei von einem `.` abgeschlossen.

Die oben gezeigte Abfrage ist die "universelle" Abfrage, die immer funktioniert, weil sie gar keine Vorgaben an das Pattern macht und auf allen drei Positionen des Triples eine Variable (bezeichnet durch das `?` vor dem Variablenname) beinhaltet. Somit gibt eine solche Abfrage alle Triples der Datenbank zurück (weil ja die Daten in einem Triple Store als Triples vorliegen). Diese Abfrage muss somit noch mit einem `LIMIT` versehen werden, das die zurückgegebene Anzahl von Resultaten beschränkt.

### Die ersten SPARQL Queries

Um im JupyterLite Notebook dieses Tutorials eine neue SPARQL Abfrage zu erstellen, erzeugen Sie eine neue Zelle für Code ("Plus-Zeichen" in der Titelzeile des aktuellen Tabs drücken und im Dropdown "Code" auswählen. Danach können Sie mit dem Python Befehl `await query(query_string, triple_store)` die Abfrage ausführen, welche als Resultat ein Pandas Dataframe zurückgibt, welches sinnvollerweise einer Variable zugewiesen wird. Das Schlüsselwort `await` ist notwendig, weil die Abfrage asynchronen Code enthält. Die Anzeige des Dataframes erfolgt mit dem Befehl `display_result(df)`, welcher dafür sorgt, dass die URI im Dataframe als klickbare Links dargestellt werden. Die dreifachen Anführungszeichen des `query_string` ermöglichen, den String über mehrere Zeilen umzubrechen und damit eine übersichtliche Darstellung der Query zu erreichen. Um die Daten aus dem Fedlex Triple Store zu beziehen, muss für `triple_store` ein "F" Charakter übergeben werden. Soll eine Query aus dem Tutorial über das SPARQL Web-Interface ausgeführt werden, kopieren Sie einfach den Teil zwischen den `"""` und fügen sie im [SPARQL Web-Interface](https://fedlex.data.admin.ch/de-CH/sparql) ein.

In [2]:
df = await query('''

SELECT * WHERE
{
    ?s ?p ?o.
}
LIMIT 10

''', 'F')

display_result(df)

,s,p,o
0,http://www.openlinksw.com/virtrdf-data-formats#default-iid,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat
1,http://www.openlinksw.com/virtrdf-data-formats#default-iid-nullable,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat
2,http://www.openlinksw.com/virtrdf-data-formats#default-iid-nonblank,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat
3,http://www.openlinksw.com/virtrdf-data-formats#default-iid-nonblank-nullable,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat
4,http://www.openlinksw.com/virtrdf-data-formats#default,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat
5,http://www.openlinksw.com/virtrdf-data-formats#default-nullable,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat
6,http://www.openlinksw.com/virtrdf-data-formats#sql-varchar,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat
7,http://www.openlinksw.com/virtrdf-data-formats#sql-varchar-nullable,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat
8,http://www.openlinksw.com/virtrdf-data-formats#sql-varchar-dt,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat
9,http://www.openlinksw.com/virtrdf-data-formats#sql-varchar-dt-nullable,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat


Diese "universelle" Abfrage ist meist etwas schwierig zu interpretieren. Eine etwas hilfreichere Abfrage kann sein, welche Typen (Klassen) von Objekten vorkommen. Da der Typ eines Elements in RDF wichtig ist, kann ganz einfach mit dem Prädikat `a` nach dem Typ gefragt werden:

In [3]:
df = await query('''

SELECT ?type WHERE
{
    ?s a ?type. # 'a' ist die Abkürzung für 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type', die immer gilt
}
LIMIT 20
''', 'F')

display_result(df)

,type
0,http://www.w3.org/2000/01/rdf-schema#Datatype
1,http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat
2,http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat
3,http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat
4,http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat
5,http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat
6,http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat
7,http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat
8,http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat
9,http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat


In dieser Form macht die Abfrage immer noch wenig Sinn, weil unter den Resultaten verschiedene die gleiche Klasse haben. Wir möchten aber ja sehen, welche verschiedenen Klassen existieren. Der Grund dafür, dass einzelne Resultate mehrmals auftauchen ist, weil diese ja aus einem Triple stammen. Weil wir nur die Variable `?type` anzeigen, sehen wir aber diese vollständigen Triples nicht (die sich unterscheiden würden). Um jedes Resultat nur einmal zu haben, kann das Schlüsselwort `DISTINCT` angefügt werden:

In [4]:
df = await query('''

SELECT DISTINCT ?type WHERE
{
    ?s a ?type. # 'a' ist die Abkürzung für 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type', die immer gilt
}
LIMIT 20
''', 'F')

display_result(df)

,type
0,http://www.w3.org/2000/01/rdf-schema#Datatype
1,http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat
2,http://www.openlinksw.com/schemas/virtrdf#QuadStorage
3,http://www.openlinksw.com/schemas/virtrdf#array-of-QuadMap
4,http://www.openlinksw.com/schemas/virtrdf#QuadMap
5,http://www.openlinksw.com/schemas/virtrdf#RdfDebugger
6,http://www.openlinksw.com/schemas/virtrdf#array-of-QuadMapFormat
7,http://www.openlinksw.com/schemas/virtrdf#QuadMapValue
8,http://www.openlinksw.com/schemas/virtrdf#array-of-QuadMapATable
9,http://www.openlinksw.com/schemas/virtrdf#array-of-QuadMapColumn


### Weitere Ressourcen zu SPARQL

- Allgemeine Einführung in SPARQL: https://jena.apache.org/tutorials/sparql.html
- Wikidata SPARQL Tutorial: https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial

## Weitere Informationen zu Linked Data

Wer vertieft in das Thema Linked Data einsteigen möchte, dem sei beispielsweise [diese Youtube Playlist](https://www.youtube.com/watch?v=ON0wf0SEPx8&list=PLoOmvuyo5UAfY6jb46jCpMoqb-dbVewxg) empfohlen.

# Die Daten auf Fedlex

## Datenmodell JOLux

Linked Data und RDF definieren die Grundmechanismen der Datenspeicherung, das eigentliche Datenmodell (auch Ontologie oder Schema genannt) muss in der jeweiligen Anwendung festgelegt werden. Das von Fedlex verwendete Datenmodell heisst **JOLux**. Dieses stammt ursprünglich aus Luxemburg und wird inzwischen von der Schweiz und Luxemburg gemeinsam weiterentwickelt. Das Datenmodell basiert auf dem [FRBR-Standard](https://de.wikipedia.org/wiki/Functional_Requirements_for_Bibliographic_Records) (Functional Requirements for Bibliographic Records), einem Entity-Relationship-Modell zur Beschreibung bibliographischer Daten, das für die Beschreibung von Rechtstexten adaptiert wurde. **JOLux** definiert, welche Art von Objekten es gibt, wie diese in Beziehung stehen können und mit welchem Vokabular (also mit welchen Prädikaten und Objekten) diese bezeichnet werden. Weitere Informationen über das Datenmodell JOLux finden Sie [hier](https://fedlex.data.admin.ch/de-CH/home/models).

## Amtliche Sammlung

In der **Amtlichen Sammlung (AS)** (Englisch: Official Compilation (OC)) des Bundesrechts werden die neuen und geänderten Erlasse, Verträge und Beschlüsse chronologisch veröffentlicht. Typischerweise haben Beschlüsse die Form eines Mantelerlasses, in denen sowohl neue Gesetze erlassen werden, als auch schon bestehende geändert oder aufgehoben werden können. Jede Veröffentlichung in der amtlichen Sammlung erhält eine eindeutige AS-Nummer (z.B. [AS 2021 654](https://www.fedlex.admin.ch/eli/oc/2021/654)). Weitere Erläuterungen zur amtlichen Sammlung sind [hier](https://www.fedlex.admin.ch/de/oc/explanations-oc) zu finden.

## Systematische Rechtssammlung

Die **Systematische Rechtssammlung (SR)** (Englisch: Consolidated Compilation (CC)) des Bundesrechts stellt die konsolidierte Fassung des aktuell gültigen Rechts basierend auf der amtlichen Sammlung dar. Jeder Gesetzestext bekommt eine SR-Nummer (z.B. [SR 101](https://www.fedlex.admin.ch/eli/cc/1999/404) für die Bundesverfassung), welche sich nicht auf eine bestimmte Version eines Gesetzes bezieht, sondern auf das Gesetz im Allgemeinen. Erscheint ein neuer Erlass in der AS der Teile des entsprechenden Gesetzes revidiert (z.B. Teile der Bundesverfassung durch [AS 2022 241](https://www.fedlex.admin.ch/eli/oc/2022/241)), so werden diese Änderungen in der systematischen Rechtssammlung konsolidiert und unter gleichbleibender SR-Nummer veröffentlicht (im Beispiel weiterhin SR 101). Eine SR-Nummer ist innerhalb des geltenden Rechts eindeutig. In Fällen in denen ein Gesetz im gesamten aufgehoben und von einem neuen Gesetz abgelöst wird (z.B. bei einer Totalrevision), kann es vorkommen, dass die SR-Nummer vom Vorgängegesetz übernommen wird. So teilt sich beispielsweise die geltende Bundesverfassung, als Ergebnis der Totalrevision von [1999](https://www.fedlex.admin.ch/eli/cc/1999/404), die SR-Nummer 101 mit ihrer Vorgängerversion von [1878](https://www.fedlex.admin.ch/eli/cc/1/1_1_1). Weitere Erläuterungen zur systematischen Rechtssammlung sind [hier](https://www.fedlex.admin.ch/de/cc/explanations-cc) zu finden.

## Aufbau der URI bei Fedlex

URI können grundsätzlich sprechend oder nicht sprechend aufgebaut sein. Sprechend heisst in diesem Zusammenhang, dass die URI schon etwas über das entsprechende Objekt kodieren. Nicht sprechende URI sind typischerweise als beliebige Folgen von Zahlen und Buchstaben aufgebaut. Die URI von Fedlex sind sprechend und richten sich nach dem europäischen [ELI-Standard](https://de.wikipedia.org/wiki/European_Legislation_Identifier) (European Legislation Identifier) zur Bezwichnung von Rechtstexten. Alle URIs beginnen mit `https://fedlex.data.admin.ch/eli/`. Die Einträge für die AS beginnen mit `https://fedlex.data.admin.ch/eli/oc/` und diejenigen für die SR mit `https://fedlex.data.admin.ch/eli/cc/`. Der weitere Aufbau der URI ist [hier](https://fedlex.data.admin.ch/de-CH/home/convention) erklärt.

## Metadaten und Triples

Mit welchen Linked Data Triples ein bestimmter Eintrag in der AS oder SR in der Datenbank beschrieben ist, lässt sich über den [Metadaten-Explorer](https://fedlex.data.admin.ch/de-CH/metadata) anzeigen. Dazu müssen die entsprechenden URIs im Metadaten-Explorer eingegeben werden.

Die URI eines AS oder SR Eintrags lässt sich über die Web-Darstellung des entsprechenden Eintrags finden, indem man auf das Ketten-Symbol direkt neben "Alles einblenden" klickt:
<br><img src="img/header_BV_URI.png" width="800px">

[In den Metadaten-Explorer eingegeben](https://fedlex.data.admin.ch/de-CH/metadata?value=https://fedlex.data.admin.ch/eli/cc/1999/404) zeigt die oben genannte URI die abgespeicherten Eigenschaften der Bundesverfassung.
<br><img src="img/metadata.png" width="800px">

Um direkt den Metadaten-Explorer Eintrag für eine bestimmte URI zu erhalten, lässt sich die URI als Parameter in der URL zum Metadaten-Explorer hinzufügen, für die Bundesverfassung also: https://fedlex.data.admin.ch/de-CH/metadata?value=https://fedlex.data.admin.ch/eli/cc/1999/404

Fedlex ist so aufgebaut, dass die Dereferenzierung einer URI (also das Anzeigen der entsprechenden Website) mit einer von der URI abweichenden URL geschieht, die sich jedoch aus der URI ableiten lässt und auf die man bei Eingabe der URI in der Browser-Adresszeile auch automatisch weitergeleitet wird:

* URI: https://fedlex.data.admin.ch/eli/cc/1999/404
* URL (Dereferenzierung der URI für Deutsch): https://www.fedlex.admin.ch/eli/cc/1999/404/de

## Prefixes

URIs sind analog zu Webadressen aufgebaut. Um die Lesbarkeit zu verbessern, können Abkürzungen, sogenannte **Prefixes** definiert werden, die häufig genutzte Teile einer URI zusammenfassen, bspw. `fedlex` als Prefix für `https://fedlex.data.admin.ch/eli/`. Aus `https://fedlex.data.admin.ch/eli/cc/1999/404` wird dann `fedlex:cc/1999/404`.

Für die weiteren Teile des Tutorials definieren wir die folgenden Prefixes:
* `fedlex` für `https://fedlex.data.admin.ch/eli/`; Namensraum für AS und SR Einträge
* `jolux` für `http://data.legilux.public.lu/resource/ontology/jolux#`; Vokabular für das JOLux Datenmodell
* `skos` für `http://www.w3.org/2004/02/skos/core#`; Externes Vokabular
* `rdf` für `http://www.w3.org/1999/02/22-rdf-syntax-ns#`; Externes Vokabular

Mehr zum Aufbau der URIs auf Fedlex finden Sie [hier](https://fedlex.data.admin.ch/de-CH/home/convention).

# Das Datenmodell erforschen und Daten abfragen

Um die gespeicherten Daten bspw. zur Bundesverfassung zu erhalten, gibt es drei Möglichkeiten:

- Die HTML Version (dereferenzierte URI) unter https://www.fedlex.admin.ch/eli/cc/1999/404/de
- Den Metadaten-Explorer unter https://fedlex.data.admin.ch/de-CH/metadata?value=https://fedlex.data.admin.ch/eli/cc/1999/404
- Die Abfrage der weiteren Daten mit SPARQL, einer Sprache zum Abfragen von Linked Data Datenbanken (so genannten Triple Stores), ähnlich zu SQL.

Die Website https://fedlex.admin.ch und alle dort angzeigten Daten basieren auf den Linked Data aus dem Triple Store. Informationen, die auf den HTML-Seiten von Fedlex zu sehen sind, sind also auch maschinenlesbar via SPARQL Abfrage verfügbar. Nachfolgend soll Schritt für Schritt das Datenmodell von Fedlex eingeführt werden und die entsprechenden SPARQL Queries aufgezeigt werden, um auf diese Daten maschinenlesbar zugreifen zu können.

## Metadaten zur Bundesverfassung

Wenn wir Infos zur Bundesverfassung wollen, können wir nach Triples suchen, in denen die Bundesverfassung als Subjekt erscheint. Dies geschieht mit folgender SPARQL Query:

In [5]:
df = await query("""

# SELECT beschreibt welche Variablen zurückgegeben werden sollen.
# Mit DISTINCT werden allfällige doppelte Ergebnisse aussortiert.

SELECT DISTINCT ?Prädikat ?Objekt WHERE {
    
    # Die Aussage beginnt mit der URI der Bundesverfassung als Subjekt, setzt das Prädikat und Objekt als Variablen und endet mit einem Punkt.
    <https://fedlex.data.admin.ch/eli/cc/1999/404> ?Prädikat ?Objekt .
} 

""", "F")

display_result(df)

,Prädikat,Objekt
0,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://data.legilux.public.lu/resource/ontology/jolux#Work
1,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://data.legilux.public.lu/resource/ontology/jolux#WorkAtOj
2,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://data.legilux.public.lu/resource/ontology/jolux#ConsolidationAbstract
3,http://data.legilux.public.lu/resource/ontology/jolux#basicAct,https://fedlex.data.admin.ch/eli/oc/1999/404
4,http://data.legilux.public.lu/resource/ontology/jolux#parliamentDraftId,1996.091
5,http://data.legilux.public.lu/resource/ontology/jolux#typeDocument,https://fedlex.data.admin.ch/vocabulary/resource-type/55
6,http://data.legilux.public.lu/resource/ontology/jolux#isRealizedBy,https://fedlex.data.admin.ch/eli/cc/1999/404/de
7,http://data.legilux.public.lu/resource/ontology/jolux#isRealizedBy,https://fedlex.data.admin.ch/eli/cc/1999/404/en
8,http://data.legilux.public.lu/resource/ontology/jolux#isRealizedBy,https://fedlex.data.admin.ch/eli/cc/1999/404/fr
9,http://data.legilux.public.lu/resource/ontology/jolux#isRealizedBy,https://fedlex.data.admin.ch/eli/cc/1999/404/it


Das Ergebniss ist eine Tabelle (eigentlich Pandas Dataframe) mit allen Prädikaten und den entsprechenden Objekten, die in allen abgespeicherten Triples mit der Bundesverfassung als Subjekt vorkommen. Als Objekte finden wir sowohl **URIs**, Objekte die dereferenzierbar und ihrerseits mit Prädikaten beschrieben sind, als auch **Literals**, Strings die eine bestimmte Information (z.B. Datum) transportieren.

Wir sehen, dass die Bundesverfassung vom `rdf:type` `jolux:ConsolidationAbstract` ist. Dieser Typ beschreibt Objekte, die einen SR-Eintrag darstellen, also ein Gesetz auf abstrakter Ebene repräsentieren.

## TaxonomyEntries: Begriffsverzeichniss

Jedem SR-Eintrag ist über das Prädikat `classifiedByTaxonomyEntry` ein Eintrag in einem [Begriffsverzeichniss](https://fedlex.data.admin.ch/vocabularies/de/) zugeordnet. Dieses dient unter anderem dazu Fachbegriffen und Konzepten eine eindeutige Bedeutung und Identität zu geben. Im Datensatz von Fedlex ist der TaxonomyEntry die zuverlässigste Quelle zum Abfragen der SR-Nummer eines SR-Eintrags. Die SR-Nummer ist mit dem TaxonomyEntry durch das Prädikat `skos:notation` verknüpft:

<br><br><img src=img/taxonomy.svg>
<br><br>

Die obige Abbildung zeigt die Verknüpfung der Bundesverfassung mit ihrer SR-Nummer repräsentiert im JoLux Datenmodell.

## Expressions: Sprachversionen der SR Einträge

Da alle Texte in der Systematischen Rechtssammlung in verschiedenen Sprachen existieren (und die URI https://fedlex.data.admin.ch/eli/cc/1999/404 der Bundesverfassung ja keinen Hinweis auf eine bestimmte Sprache gibt), müssen also innerhalb eines `jolux:ConsolidationAbstract` verschiedene Sprachversionen existieren. Diese sind vom `rdf:type` `jolux:Expression` und sind durch die Eigenschaft `jolux:isRealizedBy` mit dem sprachübergreifenden Eintrag des `jolux:ConsolidationAbstract` verknüpft.

Mit einem Klick auf die URI der deutschen Sprachversion (https://fedlex.data.admin.ch/eli/cc/1999/404/de) sehen wir, dass wir hier sowohl den Titel als auch die Abkürzung auf deutsch finden. Diese URI beschreibt nicht den deutschen Text der eigentlichen Bundesverfassung sondern repräsentiert nur die "Kopfdaten" der Bundesverfassung, also Titel und Abkürzung. Der eigentliche Inhalt ist über die [Consolidations](#Consolidations:-Versionen-der-SR-Einträge) angebunden.

<br><br><img src=img/expression.svg>
<br><br>

Ausgehend von der URI der Bundesverfassung sowie dem Wissen über TaxonomyEntries und Expressions können wir nun die Information über SR-Nummer, Titel und Abkürzung der Bundesverfassung abfragen.

In [6]:
df = await query("""

# Definition der Prefixes zur besseren Lesbarkeit
PREFIX jolux: <http://data.legilux.public.lu/resource/ontology/jolux#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT DISTINCT ?SR_Nummer ?Titel ?Abkürzung WHERE {
    
    # Die erste Aussage wählt den TaxonomyEntry der Bundesverfassung aus
    <https://fedlex.data.admin.ch/eli/cc/1999/404> jolux:classifiedByTaxonomyEntry ?TaxonomyEntry .
    # Vom TaxonomyEntry bekommen wir die SR-Nummer
    ?TaxonomyEntry skos:notation ?SR_Nummer .
    
    # Als nächstes wählen wir alle Expressions (Sprachversionen) aus die durch jolux:isRealizedBy mit der Bundesverfassung verknüpft sind.
    <https://fedlex.data.admin.ch/eli/cc/1999/404> jolux:isRealizedBy ?Expression .
    
    # Dann grenzen wir die Expressions auf diejenige mit der Sprache Deutsch ein.
    ?Expression jolux:language <http://publications.europa.eu/resource/authority/language/DEU> .
    
    # Die nächsten Aussagen fragen die gewünschen Daten ab, jeweils mit der Expression als Subjekt (möglich durch das Beenden der Aussage mit ; anstatt .)
    ?Expression jolux:title ?Titel ;
                jolux:titleShort ?Abkürzung .
} 

""", "F")

display_result(df)

,SR_Nummer,Titel,Abkürzung
0,101,Bundesverfassung der Schweizerischen Eidgenossenschaft vom 18. April 1999,BV


## Liste aller SR Einträge

Basierend auf dem was wir bereits gelernt haben können wir nun ganz einfach eine Liste aller SR-Einträge und die dazugehörigen Metadaten abfragen, indem wir die URI der Bundesverfassung durch eine Variable ersetzen die vom `rdf:type` `jolux:ConsolidationAbstract` sein soll (also einen SR Eintrag darstellt):

In [7]:
df = await query("""

# Definition von Prefixes zur besseren Lesbarkeit
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX jolux: <http://data.legilux.public.lu/resource/ontology/jolux#>

SELECT DISTINCT ?SR_Nummer ?Titel ?Abkürzung ?SR_URI WHERE {
    
    # Alle Objekte vom Typ ConsolidationAbstract auswählen
    ?SR_URI rdf:type jolux:ConsolidationAbstract .
    
    # Alle TaxonomyEntries und Expressions der SR-Einträge auswählen
    ?SR_URI jolux:classifiedByTaxonomyEntry ?TaxonomyEntry ;
            jolux:isRealizedBy ?Expression .
    
    ?TaxonomyEntry skos:notation ?SR_Nummer .
    
    ?Expression jolux:language <http://publications.europa.eu/resource/authority/language/DEU> .
    
    ?Expression jolux:title ?Titel ;
                jolux:titleShort ?Abkürzung .
} 

# Wir beschränken die Ausgabe auf die ersten 10 Einträge
LIMIT 10

""", "F")

display_result(df)

,SR_Nummer,Titel,Abkürzung,SR_URI
0,833.12,Verordnung vom 16. November 2022 über die Anpassung der Leistungen der Militärversicherung an die Lohn- und Preisentwicklung\n(MV-Anpassungsverordnung),,https://fedlex.data.admin.ch/eli/cc/2022/705
1,0.632.314.271.1,Verständigungsprotokoll vom 16. Dezember 2018 betreffend Artikel 5 (Patente) von Anhang XVII des umfassenden Wirtschaftspartnerschaftsabkommens zwischen den EFTA-Staaten und Indonesien,,https://fedlex.data.admin.ch/eli/cc/2021/716
2,0.672.919.81,Abkommen vom 3. Mai 2018 zwischen der Schweizerischen Eidgenossenschaft und der Föderativen Republik Brasilien zur Vermeidung der Doppelbesteuerung auf dem Gebiet der Steuern vom Einkommen und zur Verhinderung von Steuerhinterziehung und Steuerumgehung,,https://fedlex.data.admin.ch/eli/cc/2021/210
3,0.732.440,"Zusatzübereinkommen vom 31. Januar 1963 zum Pariser Übereinkommen vom 29. Juli 1960 über die Haftung gegenüber Dritten auf dem Gebiet der Kernenergie in der Fassung des Zusatzprotokolls vom 28. Januar 1964, des Protokolls vom 16. November 1982 und des Protokolls vom 12. Februar 2004",,https://fedlex.data.admin.ch/eli/cc/2022/45
4,0.748.127.192.85,Luftverkehrsabkommen vom 27. Februar 2019 zwischen dem Schweizerischen Bundesrat und der Regierung der Republik Costa Rica,,https://fedlex.data.admin.ch/eli/cc/2023/90
5,0.362.381.011,Notenaustausch vom 7. Oktober 2022 zwischen der Schweiz und der Europäischen Union betreffend die Übernahme der Durchführungsverordnung (EU) 2022/1409 über die detaillierten Bestimmungen für die Voraussetzungen für den Betrieb des Web-Dienstes und die für den Web-Dienst geltenden Datenschutz- und Sicherheitsvorschriften sowie über die Massnahmen für die Entwicklung und technische Umsetzung des Web-Dienstes und zur Aufhebung der Durchführungsverordnung (EU) 2021/1224 (Weiterentwicklung des Schengen-Besitzstands),,https://fedlex.data.admin.ch/eli/cc/2022/614
6,831.108,Verordnung 23 vom 12. Oktober 2022 über Anpassungen an die Lohn- und Preisentwicklung bei der AHV/IV/EO,,https://fedlex.data.admin.ch/eli/cc/2022/604
7,832.106,Verordnung des EDI vom 15. März 2022 über die Prämienregionen,,https://fedlex.data.admin.ch/eli/cc/2022/184
8,832.205.27,Verordnung 23 vom 16. November 2022 über Teuerungszulagen an Rentnerinnen und Rentner der obligatorischen Unfallversicherung,,https://fedlex.data.admin.ch/eli/cc/2022/694
9,916.401,Tierseuchenverordnung vom 27. Juni 1995 (TSV),TSV,https://fedlex.data.admin.ch/eli/cc/1995/3716_3716_3716


Der Übersicht halber wurde die Ausgabe mit dem Keyword `LIMIT` auf die ersten 10 Einträge begrenzt.

## Durchsuchen der SR-Einträge nach Abkürzung

Falls nach SR-Einträgen mit einer bestimmten Abkürzung gesucht werden soll, ist das mit einer kleinen Anpassung der vorherigen Abfrage möglich. Dazu muss lediglich die Variable `?Abkürzung` durch den gesuchten Wert (z.B. "BankV" für die Bankenverordnung) ersetzt werden. Da der gesuchte Wert ein Literal ist, muss er in Anführungszeichen gesetzt werden:

In [8]:
df = await query("""

# Definition von Prefixes zur besseren Lesbarkeit
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX jolux: <http://data.legilux.public.lu/resource/ontology/jolux#>

SELECT DISTINCT ?SR_Nummer ?Titel ?Abkürzung ?SR_URI WHERE {
    
    # Alle Objekte vom Typ ConsolidationAbstract auswählen
    ?SR_URI rdf:type jolux:ConsolidationAbstract .
    
    # Alle TaxonomyEntries und Expressions der SR-Einträge auswählen
    ?SR_URI jolux:classifiedByTaxonomyEntry ?TaxonomyEntry ;
            jolux:isRealizedBy ?Expression .
    
    ?TaxonomyEntry skos:notation ?SR_Nummer .
    
    ?Expression jolux:language <http://publications.europa.eu/resource/authority/language/DEU> .
    
    ?Expression jolux:title ?Titel ;
                jolux:titleShort "BankV" .
} 

# Wir beschränken die Ausgabe auf die ersten 10 Einträge
LIMIT 10

""", "F")

display_result(df)

,SR_Nummer,Titel,Abkürzung,SR_URI
0,952.02,"Verordnung vom 17. Mai 1972 über die Banken und Sparkassen (Bankenverordnung, BankV)",,https://fedlex.data.admin.ch/eli/cc/1972/821_832_752
1,952.02,"Verordnung vom 30. April 2014 über die Banken und Sparkassen (Bankenverordnung, BankV)",,https://fedlex.data.admin.ch/eli/cc/2014/273


Die Suche nach "BankV" ergibt zwei Gesetze die sich sowohl die Abkürzung, als auch die SR-Nummer teilen. In diesem Fall wurde die Bankenverordung 2014 einer Totalrevision unterzogen. Im Gegensatz zur Änderung einzelner Abschnitte, hat hier also ein neues Gesetz das gesamte alte Gesetz von 1972 abgelöst.

## Filtern nach geltendem Recht

In der vorherigen Abfrage haben wir gesehen, dass die Datenbank sowohl geltendes Recht, als auch aufgehobenes Recht beinhaltet. Das Datum des erstmaligen Inkraftretens ist mit `jolux:dateEntryInForce` mit dem SR-Eintrag verknüpft. Einträge die nicht mehr in Kraft sind, haben ein zusätzliches Attribut `jolux:dateNoLongerInForce`, das das Datum der Aufhebung des Gesetzes beschreibt.

Wollen wir unsere Abfrage der SR-Einträge auf das aktuell geltende Recht beschränken, können wir die Ergebnisse entsprechend filtern:

In [9]:
df = await query("""

# Definition von Prefixes zur besseren Lesbarkeit
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX jolux: <http://data.legilux.public.lu/resource/ontology/jolux#>

SELECT DISTINCT ?SR_Nummer ?Titel ?Abkürzung ?SR_URI WHERE {
    
    ?SR_URI rdf:type jolux:ConsolidationAbstract .
    
    # Auswählen des Datums des Inkrafttretens
    ?SR_URI jolux:dateEntryInForce ?datumInKraft .
    # Auswählen der SR-Einträge die vor dem heutigen Datum in Kraft getreten sind
    FILTER( ( xsd:date(?datumInKraft) <= xsd:date(now()) ) )
    
    # Auswählen des Aufhebungsdatums, falls vorhanden
    OPTIONAL { ?SR_URI jolux:dateNoLongerInForce ?datumAufhebung . }
    # Auswälen der SR-Einträge die entweder kein Aufhebungsdatum haben, oder bei denen es noch in der Zukunft liegt
    FILTER( !bound(?datumAufhebung) || xsd:date(?datumAufhebung) >= xsd:date(now()) )
    
    ?SR_URI jolux:classifiedByTaxonomyEntry ?TaxonomyEntry ;
            jolux:isRealizedBy ?Expression .
    
    ?TaxonomyEntry skos:notation ?SR_Nummer .
    
    ?Expression jolux:language <http://publications.europa.eu/resource/authority/language/DEU> .
    
    ?Expression jolux:title ?Titel ;
                jolux:titleShort ?Abkürzung .
} 

# Wir sortieren die Einträge nach Datum des Inkrafttretens in aufsteigender Reihenfolge (älteste zuerst)
ORDER BY ASC(?datumInKraft)

# Wir beschränken die Ausgabe auf die ersten 10 Einträge
LIMIT 10

""", "F")

display_result(df)

,SR_Nummer,Titel,Abkürzung,SR_URI
0,281.1,Bundesgesetz vom 11. April 1889 über Schuldbetreibung und Konkurs (SchKG),SchKG,https://fedlex.data.admin.ch/eli/cc/11/529_488_529
1,734.0,"Bundesgesetz vom 24. Juni 1902 betreffend die elektrischen Schwach- und Starkstromanlagen (Elektrizitätsgesetz, EleG)",EleG,https://fedlex.data.admin.ch/eli/cc/19/259_252_257
2,221.229.1,"Bundesgesetz vom 2. April 1908 über den Versicherungsvertrag (Versicherungsvertragsgesetz, VVG)",VVG,https://fedlex.data.admin.ch/eli/cc/24/719_735_717
3,281.51,"Verordnung vom 10. Mai 1910 betreffend die Pfändung, Arrestierung und Verwertung von Versicherungsansprüchen nach dem Bundesgesetz vom 2. April 1908 über den Versicherungsvertrag (VPAV)",VPAV,https://fedlex.data.admin.ch/eli/cc/26/205_203_198
4,220,Bundesgesetz vom 30. März 1911 betreffend die Ergänzung des Schweizerischen Zivilgesetzbuches (Fünfter Teil: Obligationenrecht),OR,https://fedlex.data.admin.ch/eli/cc/27/317_321_377
5,210,Schweizerisches Zivilgesetzbuch vom 10. Dezember 1907,ZGB,https://fedlex.data.admin.ch/eli/cc/24/233_245_233
6,211.413.1,Verordnung vom 19. Dezember 1910 betreffend die Eintragung der Eigentumsvorbehalte,EigVV,https://fedlex.data.admin.ch/eli/cc/27/215_211_275
7,281.32,Verordnung vom 13. Juli 1911 über die Geschäftsführung der Konkursämter (KOV),KOV,https://fedlex.data.admin.ch/eli/cc/27/751_749_771
8,211.423.1,Verordnung vom 30. Oktober 1917 betreffend die Viehverpfändung,VPV,https://fedlex.data.admin.ch/eli/cc/33/913_943_962
9,721.80,"Bundesgesetz vom 22. Dezember 1916 über die Nutzbarmachung der Wasserkräfte (Wasserrechtsgesetz, WRG)",WRG,https://fedlex.data.admin.ch/eli/cc/33/189_191_191


Mit dem Befehl `FILTER` können die Ergebnisse gemäss einer logischen Bedingung eingegrenzt werden. In unserem Fall werden zwei Daten vom Datentyp `xsd:date` miteinander verglichen, worbei `now()` das aktuelle Datum abruft. Ein `!` vor einer logischen Bedingung bedeuted "nicht" und negiert diese. 

Innerhalb der Umgebung `OPTIONAL` können Triples angegeben werden die optional sind, dass heisst die nicht für alle Ergebnisse existieren müssen. Hier wird sie genutzt, um das Aufhebungsdatum abzurufen, welches nur für einen Teil der SR-Einträge definiert ist. Der Ausdruck `bound()` kann dann genutzt werden, um zu überprüfen, ob für die Variable ein Wert gefunden wurde.

Mit dem Befehl `ORDER BY` wurde die Ausgabe nach Datum des Inkraftretens sortiert.

Wir sehen nun, dass das älteste noch in Kraft gebliebende Gesetz aus dem Jahre 1892 stammt. Allerdings wurde auch dieses Gesetz über die Jahre immer wieder angepasst und verändert.

## Consolidations: Versionen der SR Einträge

Ein SR Eintrag vom `rdf:type` `jolux:ConsolidationAbstract` beschreibt einen Gesetzestext in der Systematischen Rechtssammlung auf abstrakter Ebene über die Zeit seines Bestehens hinweg. Die konkrete Version des konsolidierten Gesetzestextes zu einem bestimmten Zeitpunkt ist als ein Objekt vom `rdf:type` `jolux:Consolidation` abgespeichert und mit dem Prädikat `jolux:isMemberOf` mit dem entsprechenden SR Eintrag verknüpft (Beispiel: [Bundesverfassung Version vom 13.02.2022](https://fedlex.data.admin.ch/de-CH/metadata?value=https:%2F%2Ffedlex.data.admin.ch%2Feli%2Fcc%2F1999%2F404%2F20220213)).

<br><img src=img/consolidation.svg>
<br>

Ausgehend von der URI der Bundesverfassung können wir nun eine Liste der verschiedenen Versionen abfragen:

In [10]:
df = await query("""

# Definition von Abkürzungen zur besseren Lesbarkeit
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX jolux: <http://data.legilux.public.lu/resource/ontology/jolux#>

SELECT DISTINCT ?Consolidation_Datum ?Consolidation_URI WHERE {
    
    # Alle Consolidations auswählen die zur Bundesverfassung gehören
    ?Consolidation_URI jolux:isMemberOf <https://fedlex.data.admin.ch/eli/cc/1999/404> ;
                      # Abfragen des Datums des Inkrafttretens der speziefischen Version
                      jolux:dateApplicability ?Consolidation_Datum .
} 

# Wir ordnen die Ausgabe nach absteigendem Datum (neueste zuerst)
ORDER BY DESC(?Consolidation_Datum)

# Wir beschränken die Ausgabe auf die ersten 10 Einträge
LIMIT 10

""", "F")

display_result(df)

,Consolidation_Datum,Consolidation_URI
0,2024-01-01,https://fedlex.data.admin.ch/eli/cc/1999/404/20240101
1,2022-02-13,https://fedlex.data.admin.ch/eli/cc/1999/404/20220213
2,2021-11-28,https://fedlex.data.admin.ch/eli/cc/1999/404/20211128
3,2021-03-07,https://fedlex.data.admin.ch/eli/cc/1999/404/20210307
4,2021-01-01,https://fedlex.data.admin.ch/eli/cc/1999/404/20210101
5,2020-01-01,https://fedlex.data.admin.ch/eli/cc/1999/404/20200101
6,2018-09-23,https://fedlex.data.admin.ch/eli/cc/1999/404/20180923
7,2018-01-01,https://fedlex.data.admin.ch/eli/cc/1999/404/20180101
8,2017-09-24,https://fedlex.data.admin.ch/eli/cc/1999/404/20170924
9,2017-02-12,https://fedlex.data.admin.ch/eli/cc/1999/404/20170212


## Manifestations: Dateiformate und Datenzugriff <a class="anchor" id="manifestations"></a>

Jede Version (`jolux:Consolidation`) eines SR-Eintrags (`jolux:ConsolidationAbstract`) hat dabei wiederum verschiedene Sprachversionen (`jolux:Expression`). Rufen wir die Metadaten einer solchen ab (Beispiel: [Bundesverfassung in Deutsch, Version vom 13.02.2022](https://fedlex.data.admin.ch/eli/cc/1999/404/20220213/de)), sehen wir, dass die `jolux:Expression` durch `jolux:isEmbodiedBy` wiederum mit verschiedenen Objekten vom `rdf:type` `jolux:Manifestation` verknüpft ist. Diese repräsentieren die verschiedenen Dateiformate (XML, PDF, HTML, docx, doc) in denen der Text zur Verfügung steht und zeigen ihrerseits mit dem Prädikat `jolux:isExemplifiedBy` auf den dazugehörigen Downloadlink für das Dokument.

<br><img src=img/manifestation.svg>
<br>

Mit dem Wissen über Textversionen, Sprachversionen, und Dateiformate können wir nun die vorherige Abfrage ergänzen und uns zusätzlich den Link zu dem jeweiligen PDF der verschiedenen deutschen Versionen der Bundesverfassung ausgeben lassen:

In [11]:
df = await query("""

# Definition von Abkürzungen zur besseren Lesbarkeit
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX jolux: <http://data.legilux.public.lu/resource/ontology/jolux#>

SELECT DISTINCT ?Consolidation_Datum ?Consolidation_URI ?PDF_Link WHERE {
    
    ?Consolidation_URI jolux:isMemberOf <https://fedlex.data.admin.ch/eli/cc/1999/404> ;
                      jolux:dateApplicability ?Consolidation_Datum ;
                      # Wähle alle Sprachversionen der verschiedenen Textversionen
                      jolux:isRealizedBy ?Expression .
                      
    # Reduziere die Auswahl auf die deutschen Sprachversionen
    ?Expression jolux:language <http://publications.europa.eu/resource/authority/language/DEU> ;
                # Wähle alle verfügbaren Dateiformate der Sprachversion
                jolux:isEmbodiedBy ?Manifestation .
    
    # Reduziere die Auswahl auf Einträge mit dem Datenformat PDF
    ?Manifestation jolux:format <http://publications.europa.eu/resource/authority/file-type/PDF>;
                   # Rufe den Download-Link zum entsprechenden Dokument ab
                   jolux:isExemplifiedBy ?PDF_Link

} 

# Wir ordnen die Ausgabe nach absteigendem Datum (neueste zuerst)
ORDER BY DESC(?Consolidation_Datum)

# Wir beschränken die Ausgabe auf die ersten 10 Einträge
LIMIT 10

""", "F")

display_result(df)

,Consolidation_Datum,Consolidation_URI,PDF_Link
0,2024-01-01,https://fedlex.data.admin.ch/eli/cc/1999/404/20240101,https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/1999/404/20240101/de/pdf-a/fedlex-data-admin-ch-eli-cc-1999-404-20240101-de-pdf-a.pdf
1,2022-02-13,https://fedlex.data.admin.ch/eli/cc/1999/404/20220213,https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/1999/404/20220213/de/pdf-a/fedlex-data-admin-ch-eli-cc-1999-404-20220213-de-pdf-a-4.pdf
2,2021-11-28,https://fedlex.data.admin.ch/eli/cc/1999/404/20211128,https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/1999/404/20211128/de/pdf-a/fedlex-data-admin-ch-eli-cc-1999-404-20211128-de-pdf-a-2.pdf
3,2021-03-07,https://fedlex.data.admin.ch/eli/cc/1999/404/20210307,https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/1999/404/20210307/de/pdf-a/fedlex-data-admin-ch-eli-cc-1999-404-20210307-de-pdf-a-3.pdf
4,2021-01-01,https://fedlex.data.admin.ch/eli/cc/1999/404/20210101,https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/1999/404/20210101/de/pdf-a/fedlex-data-admin-ch-eli-cc-1999-404-20210101-de-pdf-a-2.pdf
5,2020-01-01,https://fedlex.data.admin.ch/eli/cc/1999/404/20200101,https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/1999/404/20200101/de/pdf-a/fedlex-data-admin-ch-eli-cc-1999-404-20200101-de-pdf-a.pdf
6,2018-09-23,https://fedlex.data.admin.ch/eli/cc/1999/404/20180923,https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/1999/404/20180923/de/pdf-a/fedlex-data-admin-ch-eli-cc-1999-404-20180923-de-pdf-a.pdf
7,2018-01-01,https://fedlex.data.admin.ch/eli/cc/1999/404/20180101,https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/1999/404/20180101/de/pdf-a/fedlex-data-admin-ch-eli-cc-1999-404-20180101-de-pdf-a.pdf
8,2017-09-24,https://fedlex.data.admin.ch/eli/cc/1999/404/20170924,https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/1999/404/20170924/de/pdf-a/fedlex-data-admin-ch-eli-cc-1999-404-20170924-de-pdf-a.pdf
9,2017-02-12,https://fedlex.data.admin.ch/eli/cc/1999/404/20170212,https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/1999/404/20170212/de/pdf-a/fedlex-data-admin-ch-eli-cc-1999-404-20170212-de-pdf-a.pdf


## Citations und Subdivisions: Zitate zwischen SR-Einträgen<a class="anchor" id="citations"></a>

Gesetzestexte nehmen häufig Bezug aufeinander in Form von Zitaten. Auf der HTML-Seite eines SR-Eintrags lässt sich über den Link "Zitate" eine Liste aller SR-Einträge anzeigen welche den entsprechenden Text referenzieren, hier z.B. für die Bundesverfassung: https://www.fedlex.admin.ch/eli/cc/1999/404/de/quotes. 

<br><img src="img/header_BV_Zitate.png" width="800px">

Im JoLux Datenmodell sind diese Verknüpfungen als Objekte vom `rdf:type` `jolux:Citation` abgespeichert, welche sowohl auf den zitierenden (mit `jolux:citationFromLegalResource`) als auch auf den zitierten Eintrag (mit `jolux:citationToLegalResource`) zeigen:

<br><img src=img/citations.svg>
<br>

Das obige Beispiel zeigt ein [Zitat](https://fedlex.data.admin.ch/eli/cc/1977/1352_1352_1352/text/20170601/citation/3) von Artikel 10 des [Bundesgesetz vom 19. März 1976 über die internationale Entwicklungszusammenarbeit und humanitäre Hilfe](https://fedlex.data.admin.ch/eli/cc/1977/1352_1352_1352) auf die [Bundesverfassung](https://www.fedlex.admin.ch/eli/cc/1999/404/de), ausgedrückt im JoLux Datenmodell. Die Citation ist dabei nicht mit den entsprechenden SR-Einträgen direkt verknüpft sondern nur mit einem Teil davon. Untergeordnete Teile eines SR-Eintrags können entweder den enthaltenen Text oder einzelen Abschnitte und Artikel repräsentieren und sind vom `rdf:type` `jolux:LegalResourceSubdivision`. Diese untergeordneten Teile sind mit dem Prädikat `jolux:legalResourceSubdivisionIsPartOf` entweder mit einer `jolux:Consolidation` oder einem `jolux:ConsolidationAbstract` des entsprechenden SR-Eintrags verknüpft.

Eine Citation ist dabei auf der zitierenden Seite immer mit einer `jolux:LegalResourceSubdivision` einer `jolux:Consolidation` verknüpft und auf der zitierten Seite immer mit einer `jolux:LegalResourceSubdivision` eines `jolux:ConsolidationAbstract` verknüpft. Eine konkrete Version eines Gesetzes referenziert also die abstrakte und versionsübergreifende Version eines anderen Gesetzes.

Mit dem Wissen über die Räpresentation der Zitate im JoLux Datenmodell können wir nun die Information der [Liste der SR-Einträge welche die Bundesverfassung zitieren](https://www.fedlex.admin.ch/eli/cc/1999/404/de/quotes) auch mit SPARQL abfragen:

In [12]:
df = await query("""

PREFIX jolux: <http://data.legilux.public.lu/resource/ontology/jolux#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
 
SELECT DISTINCT ?Zitierendes_Gesetz_SR ?Zitierendes_Gesetz_Titel ?Zitierendes_Gesetz_Artikel ?Citation_URI WHERE {
    
    # Wähle alle Subdivision der Bundesverfassung aus
    ?Subdivision jolux:legalResourceSubdivisionIsPartOf <https://fedlex.data.admin.ch/eli/cc/1999/404> . 
    
    # Wähle alle Citations die als Ziel eine der Subdivisions angeben
    ?Citation_URI jolux:citationToLegalResource ?Subdivision . 
    # Reduziere die Auswahl auf Zitate im deutschen Text
    ?Citation_URI jolux:language <http://publications.europa.eu/resource/authority/language/DEU> .
    
    # Nun brauchen wir noch die Daten der zitierenden Texte
    # Benutze dazu die von der Citation als Quelle angegeben Subdivisions um die dazugehörigen ConsolidationAbstracts auszuwählen 
    # (LegalResourceSubdivision -> Consolidation -> ConsolidationAbstract)
    ?Citation_URI jolux:citationFromLegalResource/jolux:legalResourceSubdivisionIsPartOf/jolux:isMemberOf ?Zitierendes_Gesetz .
    
    # Wähle wie in Abschnitt 2.3.5 den Titel und die SR Nummer der zitierenden Texte über die Sprachversionen (Expressions) und TaconomyEntries aus 
    ?Zitierendes_Gesetz jolux:classifiedByTaxonomyEntry ?TaxonomyEntry ; # Wähle alle TaxonomyEntries
                        jolux:isRealizedBy ?Expression . # Wähle alle Expressions (Sprachausgaben)
    ?TaxonomyEntry skos:notation ?Zitierendes_Gesetz_SR .
    ?Expression jolux:language <http://publications.europa.eu/resource/authority/language/DEU> ;
                jolux:title ?Zitierendes_Gesetz_Titel .
                
    # Der genaue Artikel in dem zitiert wird ist in der Citation selbst beschrieben
    # Allerdings ist diese Information nicht immer verfügar weshalb wir OPTIONAL benutzen
    OPTIONAL { ?Citation_URI jolux:descriptionFrom ?Zitierendes_Gesetz_Artikel . }
    
    ### Zeige nur Einträge des geltended Rechts (analog zu Abschnitt 2.3.8)
    ?Zitierendes_Gesetz jolux:dateEntryInForce ?datumInKraft .
    FILTER( ( xsd:date(?datumInKraft) <= xsd:date(now()) ) )
    OPTIONAL { ?Zitierendes_Gesetz jolux:dateNoLongerInForce ?datumAufhebung . }
    FILTER( !bound(?datumAufhebung) || xsd:date(?datumAufhebung) >= xsd:date(now()) )
} 

ORDER BY desc(?Zitierendes_Gesetz_SR)

LIMIT 10

""", "F")

display_result(df)

,Zitierendes_Gesetz_SR,Zitierendes_Gesetz_Titel,Zitierendes_Gesetz_Artikel,Citation_URI
0,979.1,Bundesgesetz vom 4. Oktober 1991 über die Mitwirkung der Schweiz an den Institutionen von Bretton Woods,,https://fedlex.data.admin.ch/eli/cc/1992/2567_2567_2567/20021201/citation/11
1,979.1,Bundesgesetz vom 4. Oktober 1991 über die Mitwirkung der Schweiz an den Institutionen von Bretton Woods,,https://fedlex.data.admin.ch/eli/cc/1992/2567_2567_2567/20021201/citation/31
2,974.1,Bundesgesetz vom 30. September 2016 über die Zusammenarbeit mit den Staaten Osteuropas,,https://fedlex.data.admin.ch/eli/cc/2017/323/20220101/citation/11
3,974.0,Bundesgesetz vom 19. März 1976 über die internationale Entwicklungszusammenarbeit und humanitäre Hilfe,,https://fedlex.data.admin.ch/eli/cc/1977/1352_1352_1352/20220101/citation/11
4,974.0,Bundesgesetz vom 19. März 1976 über die internationale Entwicklungszusammenarbeit und humanitäre Hilfe,Art. 10Internationale Vereinbarungen,https://fedlex.data.admin.ch/eli/cc/1977/1352_1352_1352/20220101/citation/91
5,961.01,"Bundesgesetz vom 17. Dezember 2004 betreffend die Aufsicht über Versicherungsunternehmen (Versicherungsaufsichtsgesetz, VAG)",,https://fedlex.data.admin.ch/eli/cc/2005/734/20230101/citation/11
6,958.2,Verordnung vom 30. November 2018 über die Anerkennung ausländischer Handelsplätze für den Handel mit Beteiligungspapieren von Gesellschaften mit Sitz in der Schweiz,,https://fedlex.data.admin.ch/eli/cc/2018/689/20220101/citation/11
7,958.1,"Bundesgesetz vom 19. Juni 2015 über die Finanzmarktinfrastrukturen und das Marktverhalten im Effekten- und Derivatehandel (Finanzmarktinfrastrukturgesetz, FinfraG)",,https://fedlex.data.admin.ch/eli/cc/2015/853/20230101/citation/11
8,957.1,"Bundesgesetz vom 3. Oktober 2008 über Bucheffekten (Bucheffektengesetz, BEG)",,https://fedlex.data.admin.ch/eli/cc/2009/450/20230101/citation/11
9,956.1,"Bundesgesetz vom 22. Juni 2007 über die Eidgenössische Finanzmarktaufsicht (Finanzmarktaufsichtsgesetz, FINMAG)",,https://fedlex.data.admin.ch/eli/cc/2008/736/20230901/citation/11


Existiert keine Information über den Artikel in welchem zitiert wird, befindet sich das Zitat meistens im Titel, oder in der Präambel des Gesetzes, was bei Referenzen auf die Bundesverfassung besonders häufig der Fall ist.

Basierend auf dem oben beschriebenen Datenmodell können wir die obige Anfrage auch anders herum formulieren, in dem wir von der Quelle der Referenzen ausgehen und die entsprechenden Ziele abfragen. So erhalten wir eine Liste aller SR-Einträge welche von der Bundesverfassung zitiert werden:

In [13]:
df = await query("""

PREFIX jolux: <http://data.legilux.public.lu/resource/ontology/jolux#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
# Führe die Abkürzung "rdf" ein
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
 
SELECT DISTINCT ?Zitierender_Artikel ?Zitiertes_Gesetz_SR ?Zitiertes_Gesetz_Titel ?Citation_URI WHERE {
    
    # Wähle alle Consolidations (Versionen) der Bundesverfassung aus
    ?Consolidation jolux:isMemberOf <https://fedlex.data.admin.ch/eli/cc/1999/404> . 
    # Wähle alle Subdivisions der Versionen aus
    ?Subdivision jolux:legalResourceSubdivisionIsPartOf ?Consolidation .
    
    # Wähle alle Citations die als Quelle eine der Subdivisions angeben
    ?Citation_URI jolux:citationFromLegalResource ?Subdivision . 
    # Reduziere die Auswahl auf Zitate im deutschen Text
    ?Citation_URI jolux:language <http://publications.europa.eu/resource/authority/language/DEU> .
    
    # Nun brauchen wir noch die Daten der zitierten Texte
    # Benutze dazu die von der Citation als Ziel angegeben Subdivisions um die dazugehörigen ConsolidationAbstracts auszuwählen 
    ?Citation_URI jolux:citationToLegalResource/jolux:legalResourceSubdivisionIsPartOf ?Zitiertes_Gesetz .
    # Stelle sicher, dass die entsprechenden Einträge SR-Einträge sind.
    ?Zitiertes_Gesetz rdf:type jolux:ConsolidationAbstract .
    
    # Wähle wie in Abschnitt 2.3.4 den Titel und die SR Nummer der zitierenden Texte über die Sprachversionen (Expressions) aus 
    ?Zitiertes_Gesetz jolux:isRealizedBy ?Expression . # Wähle alle Expressions (Sprachausgaben)
    ?Expression jolux:language <http://publications.europa.eu/resource/authority/language/DEU> .
    ?Expression jolux:title ?Zitiertes_Gesetz_Titel ;
                jolux:historicalLegalId ?Zitiertes_Gesetz_SR .
                
    # Der genaue Artikel in dem zitiert wird ist in der Citation selbst beschrieben
    # Allerdings ist diese Information nicht immer verfügar weshalb wir OPTIONAL benutzen
    OPTIONAL { ?Citation_URI jolux:descriptionFrom ?Zitierender_Artikel . }
    
    
    ### Zeige nur Einträge des geltended Rechts (analog zu Abschnitt 2.3.7)
    ?Zitiertes_Gesetz jolux:dateEntryInForce ?datumInKraft .
    FILTER( ( xsd:date(?datumInKraft) <= xsd:date(now()) ) )
    OPTIONAL { ?Zitiertes_Gesetz jolux:dateNoLongerInForce ?datumAufhebung . }
    FILTER( !bound(?datumAufhebung) || xsd:date(?datumAufhebung) >= xsd:date(now()) )
} 

ORDER BY desc(?Zitiertes_Gesetz_SR)

LIMIT 10

""", "F")

display_result(df)

,Zitierender_Artikel,Zitiertes_Gesetz_SR,Zitiertes_Gesetz_Titel,Citation_URI
0,Art. 197Übergangsbestimmungen nach Annahme der Bundesverfassung vom 18. April 1999,831.20,Bundesgesetz vom 19. Juni 1959 über die Invalidenversicherung (IVG),https://fedlex.data.admin.ch/eli/cc/1999/404/20220213/citation/6331
1,Art. 197Übergangsbestimmungen nach Annahme der Bundesverfassung vom 18. April 1999,831.20,Bundesgesetz vom 19. Juni 1959 über die Invalidenversicherung (IVG),https://fedlex.data.admin.ch/eli/cc/1999/404/20240101/citation/6371
2,Art. 197Übergangsbestimmungen nach Annahme der Bundesverfassung vom 18. April 1999,831.10,Bundesgesetz vom 20. Dezember 1946 über die Alters- und Hinterlassenenversicherung (AHVG),https://fedlex.data.admin.ch/eli/cc/1999/404/20220213/citation/6491
3,Art. 197Übergangsbestimmungen nach Annahme der Bundesverfassung vom 18. April 1999,831.10,Bundesgesetz vom 20. Dezember 1946 über die Alters- und Hinterlassenenversicherung (AHVG),https://fedlex.data.admin.ch/eli/cc/1999/404/20240101/citation/6531
4,Art. 196Übergangsbestimmungen gemäss Bundesbeschluss vom 18. Dezember 1998 über eine neue Bundesverfassung,822.11,"Bundesgesetz vom 13. März 1964 über die Arbeit in Industrie, Gewerbe und Handel (Arbeitsgesetz, ArG)",https://fedlex.data.admin.ch/eli/cc/1999/404/20220213/citation/5681
5,Art. 196Übergangsbestimmungen gemäss Bundesbeschluss vom 18. Dezember 1998 über eine neue Bundesverfassung,822.11,"Bundesgesetz vom 13. März 1964 über die Arbeit in Industrie, Gewerbe und Handel (Arbeitsgesetz, ArG)",https://fedlex.data.admin.ch/eli/cc/1999/404/20240101/citation/5721
6,Art. 196Übergangsbestimmungen gemäss Bundesbeschluss vom 18. Dezember 1998 über eine neue Bundesverfassung,741.01,Strassenverkehrsgesetz vom 19. Dezember 1958 (SVG),https://fedlex.data.admin.ch/eli/cc/1999/404/20220213/citation/5431
7,Art. 196Übergangsbestimmungen gemäss Bundesbeschluss vom 18. Dezember 1998 über eine neue Bundesverfassung,741.01,Strassenverkehrsgesetz vom 19. Dezember 1958 (SVG),https://fedlex.data.admin.ch/eli/cc/1999/404/20240101/citation/5471
8,Art. 196Übergangsbestimmungen gemäss Bundesbeschluss vom 18. Dezember 1998 über eine neue Bundesverfassung,641.81,"Bundesgesetz vom 19. Dezember 1997 über eine leistungsabhängige Schwerverkehrsabgabe (Schwerverkehrsabgabegesetz, SVAG)",https://fedlex.data.admin.ch/eli/cc/1999/404/20220213/citation/5441
9,Art. 196Übergangsbestimmungen gemäss Bundesbeschluss vom 18. Dezember 1998 über eine neue Bundesverfassung,641.81,"Bundesgesetz vom 19. Dezember 1997 über eine leistungsabhängige Schwerverkehrsabgabe (Schwerverkehrsabgabegesetz, SVAG)",https://fedlex.data.admin.ch/eli/cc/1999/404/20240101/citation/5481


## Impacts: Änderungen der SR durch AS-Erlasse<a class="anchor" id="citations"></a>

In Erlassen der Amtlichen Sammlung (AS) können sowohl neue Gesetze erlassen werden, als auch Teile von schon bestehenden Gesetzen geändert oder aufgehoben werden. Ein neuer Eintrag in der AS führt dementsprechend im Normalfall zu Änderungen in bestehenden SR-Einträgen, die in einer neuen Version (Consolidation) konsolidiert werden. Die bisherigen Änderungen der Bundesverfassung beispielsweise und die dazugehörigen Erlasse in der AS können in der HTML-Ansicht über den Link [Änderungen](https://www.fedlex.admin.ch/eli/cc/1999/404/de/changes) sichtbar gemacht werden. 

<br><img src="img/header_BV_Änderungen.png" width="800px">

AS-Einträge sind im JoLux Datenmodell durch Objekte vom `rdf:type` `jolux:Act` repräsentiert. Die Änderungen oder Auswirkungen eines AS-Eintrags auf die SR sind durch Objekte vom `rdf:type` `jolux:LegalResourceImpact` repräsentiert. Ähnlich wie die `jolux:Citation` Objekte sind diese dabei sowohl auf der Quellseite (durch `jolux:impactFromLegalResource`) als auch auf der Zielseite (durch `jolux:impactToLegalResource`) mit einer `jolux:LegalResourceSubdivision` verknüpft. Dabei zeigt der Impact auf der Quellseite immer auf eine Subdivision eines `jolux:Act` und auf der Zielseite auf eine Subdivision eines `jolux:ConsolidationAbstract`.

<br><img src=img/impact.svg>
<br>

Das obige Beispiel zeigt einen [Impact](https://fedlex.data.admin.ch/eli/cc/1999/404/impact/226) auf den Art. 196 der [Bundesverfassung](https://www.fedlex.admin.ch/eli/cc/1999/404/de) durch den [Bundesbeschluss über die neue Finanzordnung 2021 (AS 2019 769)](https://www.fedlex.admin.ch/eli/oc/2019/137/de) ausgedrückt im JoLux Datenmodell. Informationen wie die Art der Änderung (Inkrafttreten, Änderung, Berichtigung oder Aufhebung), die geänderten Artikel oder der Teil des Beschlusses, welcher die Änderung bewirkt, sind als Atrribute den Impacts oder den Subdivisions (je nach Alter der Gesetze) zugeordnet.

Mit dem Wissen über Impacts können wir nun die Änderungen der Bundesverfassung auch über SPARQL abfragen:

In [14]:
df = await query("""

PREFIX jolux: <http://data.legilux.public.lu/resource/ontology/jolux#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
 
SELECT DISTINCT ?Typ ?Beschluss_Datum ?Artikel_geändert ?Beschluss ?AS_URI ?Impact_URI WHERE { 
    
    # Wähle alle Subdivisions der Bundesverfassung
    ?SRsubdivision jolux:legalResourceSubdivisionIsPartOf <https://fedlex.data.admin.ch/eli/cc/1999/404> . 
    
    # Wähle all Impacts die als Ziel eine Subdivisions der Bundesverfassung angeben, sowie die Eigenschaften der Impacts
    ?Impact_URI jolux:impactToLegalResource ?SRsubdivision ;
            jolux:language <http://publications.europa.eu/resource/authority/language/DEU> ; # Nur Deutsche Versionen
            jolux:legalResourceImpactHasType/skos:prefLabel ?Typ ; # Info über die Art der Änderung
            jolux:impactFromLegalResource/jolux:legalResourceSubdivisionIsPartOf ?AS_URI . # Auswählen der Quellen der Änderungen (AS-Einträge)
    FILTER(lang(?Typ) = 'de')  # Nur die deutschen Bezeichnungen
    
    # Impacts in der Datenbank können von verschiedenen Datenquellen kommen. Hier wollen wir eine bestimmte auswählen.
    ?Impact_URI jolux:informationSource ?source .
    FILTER(?source = <https://fedlex.data.admin.ch/vocabulary/information-source/data-from-mutation>)
    
    # Abfrage der Info welcher Teil des Bundesbeschlusses die Änderung beinhaltet (Nicht alle Impacts haben dieses Atribut deshalb OPTIONAL)
    OPTIONAL { ?AS_URI jolux:dateDocument ?Beschluss_Datum . }
    OPTIONAL { ?Impact_URI jolux:impactFromLegalResourceComment ?Beschluss . FILTER(lang(?Beschluss)='de') . }
    
    # Abfrage der Info welche Artikel die Änderung betrifft (Nicht alle Impacts haben dieses Atribut deshalb OPTIONAL)
    OPTIONAL { ?Impact_URI jolux:impactToLegalResourceComment ?Artikel_geändert . FILTER(lang(?Artikel_geändert)='de') . }
     
} ORDER BY desc(?Typ) ?Beschluss_Datum

""", "F")

display_result(df)

,Typ,Beschluss_Datum,Artikel_geändert,Beschluss,AS_URI,Impact_URI
0,Änderungen,1999-10-08,"Art. 29a, 122, 123; Tit. vor Art. 188; Art. 188-191, 191a-191c","BB 08.10.1999 (I, II)",https://fedlex.data.admin.ch/eli/oc/2002/467,https://fedlex.data.admin.ch/eli/cc/1999/404/impact/34
1,Änderungen,2001-06-22,"Art. 126, 159",BB 22.06.2001 (I),https://fedlex.data.admin.ch/eli/oc/2002/51,https://fedlex.data.admin.ch/eli/cc/1999/404/impact/19
2,Änderungen,2001-10-05,"Art. 196, 197",Volksabstimmung 03.03.2002,https://fedlex.data.admin.ch/eli/oc/2002/156,https://fedlex.data.admin.ch/eli/cc/1999/404/impact/25
3,Änderungen,2002-10-04,"Art. 138, 139b, 140, 141, 141a",BB 04.10.2002 (I),https://fedlex.data.admin.ch/eli/oc/2003/296,https://fedlex.data.admin.ch/eli/cc/1999/404/impact/47
4,Änderungen,2003-06-20,Art. 123a,Volksabstimmung 08.02.2004,https://fedlex.data.admin.ch/eli/oc/2004/272,https://fedlex.data.admin.ch/eli/cc/1999/404/impact/55
5,Änderungen,2003-10-03,"Art. 5a, 42, 43a, 46-48, 48a, 58, 60, 62, 75a, 112, 112a-112c, 123, 128, 132, 135, 196, 197","BB 03.10.2003 (I, II)",https://fedlex.data.admin.ch/eli/oc/2007/785,https://fedlex.data.admin.ch/eli/cc/1999/404/impact/91
6,Änderungen,2004-03-19,"Art. 128, 130",BB 19.03.2004 (I),https://fedlex.data.admin.ch/eli/oc/2006/184,https://fedlex.data.admin.ch/eli/cc/1999/404/impact/83
7,Änderungen,2005-06-17,Art. 197,Volksabstimmung 27.11.2005,https://fedlex.data.admin.ch/eli/oc/2006/23,https://fedlex.data.admin.ch/eli/cc/1999/404/impact/67
8,Änderungen,2005-12-16,"Art. 61a, 62, 63, 63a, 64, 64a, 65-67",BB 16.12.2005,https://fedlex.data.admin.ch/eli/oc/2006/468,https://fedlex.data.admin.ch/eli/cc/1999/404/impact/73
9,Änderungen,2008-06-13,Art. 123b,Volksabstimmung 30.11.2008,https://fedlex.data.admin.ch/eli/oc/2009/82,https://fedlex.data.admin.ch/eli/cc/1999/404/impact/97


Die Sprache von String-Literals kann durch einen Language-Tag gekennzeichnet sein. Ist, wie in der obigen Abfrage, ein Attribut mehrmals belegt mit Literals in unterschiedlichen Sprachen, kann mit der `FILTER()` Funktion und der `lang()` Funktion die Auswahl auf die gewünschte Sprache reduziert werden. Die Funktion `lang()` gibt dabei, falls existent, den Language-Tag des Literals zurück.

Bei einem Vergleich der Ergebnisse mit der [HTML-Ansicht](https://www.fedlex.admin.ch/eli/cc/1999/404/de/changes) fällt auf, dass die Änderungen ab 2020 in den Ergebnissen der SPARQL-Abfrage fehlen. Das liegt daran, dass es für Impacts in den Daten mehrere Datenquellen gibt ([Geschäftsstaende](https://fedlex.data.admin.ch/vocabulary/information-source/data-from-geschaeftsstaende), [Mutation](https://fedlex.data.admin.ch/vocabulary/information-source/data-from-mutation) und [Legiconso](https://fedlex.data.admin.ch/vocabulary/information-source/data-from-legiconso)) und einfachheitshalber in der obigen Abfrage nur Impacts die aus einer bestimmten Quelle ([Mutation](https://fedlex.data.admin.ch/vocabulary/information-source/data-from-mutation)) stammen abgerufen wurden während, die neueren Impacts lediglich in der Quelle [Legiconso](https://fedlex.data.admin.ch/vocabulary/information-source/data-from-legiconso) verfügbar sind. Eine komplette Liste aller Änderungen bedarf daher die Zusammenführung der Informationen aus den verschiedenen Quellen und Speicherorten, was in der nachfolgenden Abfrage durchgeführt wird:

**Die nachfolgende Abfrage geht über den Rahmen dieses einführenden Tutorials hinaus und wird daher hier nicht weiter erläutert:**

In [15]:
df = await query("""

PREFIX jolux: <http://data.legilux.public.lu/resource/ontology/jolux#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

# SAMPLE, MAX und GROUP_CONCAT sind Aggregierfunctionen die über alle Listeneinträge angewandt werden die in den GROUP BY variablen gleiche Werte haben
SELECT (SAMPLE(?Typ) AS ?Typ) ?Beschluss (SAMPLE(?Inkrafttreten) AS ?Inkrafttreten) (GROUP_CONCAT(?Art_geändert; SEPARATOR=', ') AS ?Artikel_geändert) ?AS_Nr 
?AS_Titel (MAX(?AS_Artikel) AS ?AS_Artikel) ?AS_URI (GROUP_CONCAT(?Impact_URI; SEPARATOR='    ') AS ?Impact_URIs)  WHERE { 
    
    # Wähle alle Subdivisions der Bundesverfassung
    ?SRsubdivision jolux:legalResourceSubdivisionIsPartOf <https://fedlex.data.admin.ch/eli/cc/1999/404> .
    
    # Wähle all Impacts die als Ziel eine Subdivisions der Bundesverfassung angeben, sowie die Eigenschaften der Impacts
    ?Impact_URI jolux:impactToLegalResource ?SRsubdivision ;
            jolux:language <http://publications.europa.eu/resource/authority/language/DEU> ; # Nur Deutsche Versionen
            jolux:legalResourceImpactHasType/skos:prefLabel ?Typ ; # Info über die Art der Änderung
            jolux:legalResourceImpactHasDateEntryInForce ?Inkrafttreten ; # Datum des Inkrafttreten der Änderung
            jolux:impactFromLegalResource/jolux:legalResourceSubdivisionIsPartOf ?AS_URI ; # Auswählen der Quellen der Änderungen (AS-Einträge)
            jolux:informationSource ?source . # Abfragen aus welcher Datenquelle der Impact stammt
    FILTER(lang(?Typ) = 'de') # Nur die deutschen Bezeichnungen
     
    # Informationen zu Beschlussdatum der AS-Einträgs sowie Titel der AS-Einträge
    ?AS_URI jolux:dateDocument ?Beschluss ;
            jolux:isRealizedBy ?AS_Expression .       
    ?AS_Expression jolux:language <http://publications.europa.eu/resource/authority/language/DEU> ;
                   jolux:title ?AS_Titel .
                   
    # Ältere AS haben die AS Nummer als Attribut gespeichert               
    OPTIONAL { ?AS_Expression jolux:historicalLegalId ?AS_Nr_ . }
    # Für neuere AS-Einträge muss diese erst zusammengesetzt werden aus Veröffentlichungsjahr und Veröffentlichungssequenz
    OPTIONAL { ?AS_Expression jolux:memorialName ?AS_Nr_Teil1 . }
    OPTIONAL { ?AS_Expression jolux:memorialYear ?AS_Nr_Teil2 . }
    OPTIONAL { ?AS_URI jolux:sequenceInTheYearOfPublication ?AS_Nr_Teil3 . }
    BIND(IF(bound(?AS_Nr), ?AS_Nr_, CONCAT(str(?AS_Nr_Teil1), ' ', str(?AS_Nr_Teil2), ' ', str(?AS_Nr_Teil3))) as ?AS_Nr)
    
    # Abfrage der Info welcher Teil des AS-Eintrags die Änderung auslöst
    # Für ältere Impacts ist diese Info ein Attribut von Impacts aus der Quelle Geschäftsende
    OPTIONAL { 
    ?Impact_URI jolux:informationSource <https://fedlex.data.admin.ch/vocabulary/information-source/data-from-geschaeftsstaende> .
    ?Impact_URI jolux:impactFromLegalResourceComment ?AS_Artikel1 . FILTER(lang(?AS_Artikel1)='de') . 
    }
    # Für neuere Impacts aus der Quelle Legiconso ist die Info ein Attribut der Quellseitigen Subdivision
    OPTIONAL { ?Impact_URI jolux:impactFromLegalResource/jolux:legalResourceSubdivisionHasSubdivisionIdentificationDetail/jolux:legalResourceSubdivisionDetailId ?AS_Artikel2 .}
    BIND ( IF(?source in (<https://fedlex.data.admin.ch/vocabulary/information-source/data-from-geschaeftsstaende>,<https://fedlex.data.admin.ch/vocabulary/information-source/data-from-mutation>), ?AS_Artikel1, ?AS_Artikel2)
        AS ?AS_Artikel )
    
    # Abfrage der Info welcher Teil des Bundesbeschlusses die Änderung beinhaltet
    # Für ältere Impacts ist diese Info ein Attribut von Impacts aus der Quelle Mutation
    OPTIONAL { 
    ?Impact_URI jolux:informationSource <https://fedlex.data.admin.ch/vocabulary/information-source/data-from-mutation> .
    ?Impact_URI jolux:impactToLegalResourceComment ?geändert1 . FILTER(lang(?geändert1)='de') . 
    }
    # Für neuere Impacts aus der Quelle Legiconso ist die Info ein Attribut der Zielseitigen Subdivision
    OPTIONAL { ?SRsubdivision jolux:legalResourceSubdivisionHasSubdivisionIdentificationDetail/jolux:legalResourceSubdivisionDetailId ?geändert2 . } # Not all SR Subdivisions have that Info
    BIND ( IF(?source in (<https://fedlex.data.admin.ch/vocabulary/information-source/data-from-geschaeftsstaende>,<https://fedlex.data.admin.ch/vocabulary/information-source/data-from-mutation>), ?geändert1, ?geändert2) AS ?Art_geändert )
     
} 

# Listeneinträge welche über diese variablen die gleichen Werte haben werden mit den im SELECT-Statement angegebenen Funktionen aggregiert
GROUP BY ?Beschluss ?AS_Nr ?AS_Titel ?AS_URI ?Artikel_geändert

ORDER BY desc(?Beschluss)

""", "F")

display_result(df)

,Typ,Beschluss,Inkrafttreten,Artikel_geändert,AS_Nr,AS_Titel,AS_Artikel,AS_URI,Impact_URIs
0,Änderungen,2022-02-13,2022-02-13,"118, 197, 41",AS 2022 241,Eidgenössische Volksinitiative «Ja zum Schutz der Kinder und Jugendlichen vor Tabakwerbung (Kinder und Jugendliche ohne Tabakwerbung)»,,https://fedlex.data.admin.ch/eli/oc/2022/241,https://fedlex.data.admin.ch/eli/oc/2022/241/legal-analysis/LegalResourceImpact/2 https://fedlex.data.admin.ch/eli/oc/2022/241/legal-analysis/LegalResourceImpact/1 https://fedlex.data.admin.ch/eli/oc/2022/241/legal-analysis/LegalResourceImpact/3
1,Änderungen,2021-12-17,2024-01-01,130,AS 2023 91,Bundesbeschluss über die Zusatzfinanzierung der AHV durch eine Erhöhung der Mehrwertsteuer,I,https://fedlex.data.admin.ch/eli/oc/2023/91,https://fedlex.data.admin.ch/eli/oc/2023/91/legal-analysis/LegalResourceImpact/1
2,Änderungen,2021-11-28,2021-11-28,"117b, 197",AS 2022 240,Eidgenössische Volksinitiative «Für eine starke Pflege (Pflegeinitiative)»,,https://fedlex.data.admin.ch/eli/oc/2022/240,https://fedlex.data.admin.ch/eli/oc/2022/240/legal-analysis/LegalResourceImpact/1 https://fedlex.data.admin.ch/eli/oc/2022/240/legal-analysis/LegalResourceImpact/2
3,Änderungen,2020-06-19,2021-03-07,"10a, 197",AS 2021 310,Bundesbeschluss zur Volksinitiative «Ja zum Verhüllungsverbot»,II,https://fedlex.data.admin.ch/eli/oc/2021/310,https://fedlex.data.admin.ch/eli/oc/2021/310/legal-analysis/LegalResourceImpact/2 https://fedlex.data.admin.ch/eli/oc/2021/310/legal-analysis/LegalResourceImpact/1
4,Änderungen,2018-03-13,2018-09-23,Art. 88,AS 2019 91,"Bundesbeschluss über die Velowege sowie die Fuss- und Wanderwege (direkter Gegenentwurf zur Volksinitiative «Zur Förderung der Velo-, Fuss- und Wanderwege [Velo-Initiative]»)",I,https://fedlex.data.admin.ch/eli/oc/2019/91,https://fedlex.data.admin.ch/eli/cc/1999/404/impact/214 https://fedlex.data.admin.ch/eli/cc/1999/404/impact/217
5,Änderungen,2017-06-16,2021-01-01,Art. 196,AS 2019 137,Bundesbeschluss über die neue Finanzordnung 2021,I,https://fedlex.data.admin.ch/eli/oc/2019/137,https://fedlex.data.admin.ch/eli/cc/1999/404/impact/223 https://fedlex.data.admin.ch/eli/cc/1999/404/impact/226
6,Änderungen,2017-03-14,2017-09-24,Art. 104a,AS 2017 752,Bundesbeschluss über die Ernährungssicherheit (direkter Gegenentwurf zur Volksinitiative «Für Ernährungssicherheit»),I,https://fedlex.data.admin.ch/eli/oc/2017/752,https://fedlex.data.admin.ch/eli/cc/1999/404/impact/202 https://fedlex.data.admin.ch/eli/cc/1999/404/impact/205
7,Änderungen,2016-09-30,2017-02-12,Art. 38,AS 2017 259,Bundesbeschluss über die erleichterte Einbürgerung von Personen der dritten Ausländergeneration,I,https://fedlex.data.admin.ch/eli/oc/2017/259,https://fedlex.data.admin.ch/eli/cc/1999/404/impact/196 https://fedlex.data.admin.ch/eli/cc/1999/404/impact/199
8,Änderungen,2016-09-30,2018-01-01,"Art. 83, 85a, 86, 87b, 131, 196",AS 2017 751,Bundesbeschluss über die Schaffung eines Fonds für die Nationalstrassen und den Agglomerationsverkehr,I,https://fedlex.data.admin.ch/eli/oc/2017/751,https://fedlex.data.admin.ch/eli/cc/1999/404/impact/208 https://fedlex.data.admin.ch/eli/cc/1999/404/impact/220 https://fedlex.data.admin.ch/eli/cc/1999/404/impact/211
9,Änderungen,2014-12-12,2015-06-14,Art. 119,AS 2015 541,Bundesbeschluss über die Änderung der Verfassungsbestimmung zur Fortpflanzungsmedizin und Gentechnologie im Humanbereich,,https://fedlex.data.admin.ch/eli/oc/2015/541,https://fedlex.data.admin.ch/eli/cc/1999/404/impact/184 https://fedlex.data.admin.ch/eli/cc/1999/404/impact/187


# Impressum

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons Lizenzvertrag" style="border-width:0" src="https://mirrors.creativecommons.org/presskit/buttons/88x31/svg/by-sa.svg" /></a><br />Dieses Werk ist lizenziert unter einer <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Namensnennung - Weitergabe unter gleichen Bedingungen 4.0 International Lizenz</a>.

Autoren: 

- [Jurek Müller](mailto:jurek.mueller@bfh.ch)
- [Benedikt Hitz-Gamper](mailto:benedikt.hitz@bfh.ch)